In [3]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.models

In [4]:
images = torch.load("/kaggle/working/image.pt")

In [5]:
labels = torch.load("/kaggle/working/labels.pt")

temp = []
for i,label in enumerate(labels):
    
    for j in range(7):
        if(int(label[j])==1):
            temp.append(torch.tensor(j))
labels = torch.stack(temp)

In [6]:
print(images.shape)
print(labels.shape)

torch.Size([10015, 3, 299, 299])
torch.Size([10015])


In [7]:
#Model definition
from torchvision.models import inception_v3,Inception_V3_Weights
weights = Inception_V3_Weights.IMAGENET1K_V1
model = inception_v3(weights = weights)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(model.fc.in_features,7)

In [8]:
# Test the model on a random Batch of 10 images to check if forward pass works

batch_images = torch.randn(10,3,299,299)
model.eval()
model(batch_images).shape

torch.Size([10, 7])

## Shape of (10,7) confirms that model is working properly

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
images.to(device)
labels.to(device)

tensor([1, 1, 1,  ..., 4, 1, 1], device='cuda:0')

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(images,labels,train_size = 0.9,shuffle=True,random_state=100)

In [12]:
num_labels_per_class = torch.bincount(y_train)
total_labels = len(labels)
class_weights = torch.tensor([total_labels / count for count in num_labels_per_class], dtype=torch.float32)
class_weights /= 7
class_weights = class_weights.to(device)
print(class_weights)

tensor([ 1.4350,  0.2368,  3.1514,  4.7690,  1.4569, 12.8893, 11.1775],
       device='cuda:0')


In [18]:
import torch.optim as optim

model = model.to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights) #using weighted cross entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [24]:
patience = 4
best_class_accuracy = 0.0
best_model_state = None
best_epoch = 0
num_epochs = 20  


for epoch in range(1,num_epochs+1):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        outputs = outputs.to(device)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
        
    
    print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # Evaluate the model on the test set after each epoch
    model.eval()
    with torch.no_grad():
        class_correct = np.zeros(7)  
        class_total = np.zeros(7)    
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            
            
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == batch_y).squeeze()
            for i in range(batch_y.size(0)):
                label = batch_y[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
        
        class_accuracy = class_correct / class_total  # Calculate accuracy for each class
        average_class_accuracy = np.mean(class_accuracy)  # Calculate average class-wise accuracy
        
        print(f'Average class-wise accuracy of the model on the test set after epoch {epoch}: {average_class_accuracy:.2f}%')
        if average_class_accuracy > best_class_accuracy:
            best_class_accuracy = average_class_accuracy
            best_model_state = model.state_dict()
            best_epoch = epoch
        else:
            if epoch - best_epoch >= patience:
                print(f'Early stopping triggered at epoch {epoch}')
                break

Epoch [1/20], Loss: 1.0835
Average class-wise accuracy of the model on the test set after epoch 1: 0.61%
Epoch [2/20], Loss: 2.0977
Average class-wise accuracy of the model on the test set after epoch 2: 0.61%
Epoch [3/20], Loss: 0.9219
Average class-wise accuracy of the model on the test set after epoch 3: 0.60%
Epoch [4/20], Loss: 0.9708
Average class-wise accuracy of the model on the test set after epoch 4: 0.58%
Epoch [5/20], Loss: 1.7373
Average class-wise accuracy of the model on the test set after epoch 5: 0.61%
Epoch [6/20], Loss: 1.2146
Average class-wise accuracy of the model on the test set after epoch 6: 0.56%
Epoch [7/20], Loss: 0.3734
Average class-wise accuracy of the model on the test set after epoch 7: 0.62%
Epoch [8/20], Loss: 1.3605
Average class-wise accuracy of the model on the test set after epoch 8: 0.64%
Epoch [9/20], Loss: 0.5639
Average class-wise accuracy of the model on the test set after epoch 9: 0.61%
Epoch [10/20], Loss: 1.2336
Average class-wise accuracy

In [26]:
model.load_state_dict(best_model_state)

<All keys matched successfully>

In [27]:
 
while(True):
    model.eval()
    with torch.no_grad():
        class_correct = np.zeros(7)  # Array to store the number of correct predictions for each class
        class_total = np.zeros(7)    # Array to store the total number of samples for each class
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.cuda(), batch_y.cuda()
            outputs = model(batch_X)
            
            # If the model returns a tuple (like Inception), use the first element
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == batch_y).squeeze()
            for i in range(batch_y.size(0)):
                label = batch_y[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
        
        class_accuracy = class_correct / class_total  # Calculate accuracy for each class
        average_class_accuracy = np.mean(class_accuracy)
        print(class_accuracy)
        print(average_class_accuracy)
        break# Calculate average class-wise accuracy

[0.57171515 0.48369475 0.73127753 0.75333333 0.69653768 0.96396396
 0.96875   ]
0.73846748664363


In [28]:
torch.save(model.state_dict(), 'best_model.pth')